In [1]:
import numpy as np
import pandas as pd

### Sample Submission

In [2]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [3]:
train = pd.read_csv('./sales_train_v2.csv')
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
test = pd.read_csv('./test.csv')
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


### Checking Nulls

In [6]:
# null check
train.item_cnt_day.isnull().sum()

0

In [17]:
print(train.date[0].split('.')[1])
print(train.date[0].split('.')[2])

01
2013


In [18]:
train['month'] = [date.split('.')[1] for date in train.date]
train['year'] = [date.split('.')[2] for date in train.date]

### Filtering+Aggregation of Oct 2015 Data for prev month baseline model

In [24]:
train_oct_2015 = train[(train.month=='10') & (train.year=='2015')]
print(train_oct_2015.month.unique())
print(train_oct_2015.year.unique())
train_oct_2015.head()

['10']
['2015']


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year
2882335,23.10.2015,33,45,13315,649.0,1.0,10,2015
2882336,05.10.2015,33,45,13880,229.0,1.0,10,2015
2882337,02.10.2015,33,45,13881,659.0,1.0,10,2015
2882338,12.10.2015,33,45,13881,659.0,1.0,10,2015
2882339,04.10.2015,33,45,13923,169.0,1.0,10,2015


In [32]:
train_oct_2015_agg = train_oct_2015.groupby(['shop_id','item_id'])[['item_cnt_day']].sum()
train_oct_2015_agg = train_oct_2015_agg.reset_index()
train_oct_2015_agg.head()

,shop_id,item_id,item_cnt_day
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0


In [36]:
print(len(train_oct_2015_agg))
print(len(train_oct_2015_agg.drop_duplicates()))

31531
31531


In [37]:
train_oct_2015_agg.item_cnt_day = np.clip(train_oct_2015_agg.item_cnt_day, 0, 20)

### Merging to get all combinations of Shops and items in Test dataset

In [39]:
test_preds = pd.merge(test, train_oct_2015_agg, how='left', left_on=['shop_id', 'item_id'], right_on=['shop_id', 'item_id'])
print(len(test))
print(len(test_preds))

214200
214200


In [41]:
test_preds.item_cnt_day.isnull().sum()

185520

In [42]:
test_preds.item_cnt_day = test_preds.item_cnt_day.fillna(0)

In [43]:
test_preds.item_cnt_day.isnull().sum()

0

In [44]:
test_preds.head()

,ID,shop_id,item_id,item_cnt_day
0,0,5,5037,0.0
1,1,5,5320,0.0
2,2,5,5233,1.0
3,3,5,5232,0.0
4,4,5,5268,0.0


In [45]:
submission = pd.DataFrame({'ID':test_preds.ID,'item_cnt_month':test_preds.item_cnt_day})
submission.head()

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,1.0
3,3,0.0
4,4,0.0


### LB 1.16777 Score

In [46]:
submission.to_csv('./baseline.csv', sep=',', encoding='utf-8', index=False)